In [1]:
import mysql.connector
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

In [2]:
pd.set_option("display.max_columns", None)

Load data from MySQL

In [3]:
# Event data
conn = mysql.connector.connect(host='localhost', user='root', database='ufc', password = '1003Yann!')

cursor = conn.cursor()

query = ("SELECT * FROM events")

cursor.execute(query)

columns = [desc[0] for desc in cursor.description]

rows = cursor.fetchall()

df_events = pd.DataFrame(rows, columns=columns)

cursor.close()
conn.close()

In [4]:
# Stats data
conn = mysql.connector.connect(host='localhost', user='root', database='ufc', password = '1003Yann!')

cursor = conn.cursor()

query = ("SELECT * FROM stats")

cursor.execute(query)

columns = [desc[0] for desc in cursor.description]

rows = cursor.fetchall()

df_stats = pd.DataFrame(rows, columns=columns)

cursor.close()
conn.close()

Preview datasets

In [5]:
df_events.head()


,id,event_date,event_name,fighter_blue,fighter_red,knockdowns_blue,knockdowns_red,sig_attempts_blue,sig_attempts_red,sig_strikes_blue,sig_strikes_red,total_strikes_attempts_blue,total_strikes_attempts_red,total_strikes_blue,total_strikes_red,sub_attempts_blue,sub_attempts_red,takedowns_blue,takedowns_red,takedown_attempts_blue,takedown_attempts_red,control_time_blue,control_time_red,round,time,weight_class,win_method,winner
0,1,1994-03-11,UFC 2: No Way Out,Remco Pardoel,Royce Gracie,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,2,0,0,1,91,Open Weight,Submission,Royce Gracie
1,2,1994-03-11,UFC 2: No Way Out,Ray Wizard,Patrick Smith,0,0,1,1,1,1,2,1,2,1,0,1,0,0,0,1,0,0,1,58,Open Weight,Submission,Patrick Smith
2,3,1994-03-11,UFC 2: No Way Out,David Levicki,Johnny Rhodes,0,0,5,17,4,11,102,86,95,74,0,0,0,1,0,1,0,0,1,733,Open Weight,KO/TKO,Johnny Rhodes
3,4,1994-03-11,UFC 2: No Way Out,Sean Daugherty,Scott Morris,0,0,4,1,0,1,5,2,1,2,0,1,0,1,0,1,0,0,1,20,Open Weight,Submission,Scott Morris
4,5,1994-03-11,UFC 2: No Way Out,Robert Lucarelli,Orlando Wiet,0,0,6,12,2,8,6,15,2,11,1,0,1,0,1,0,0,0,1,170,Open Weight,KO/TKO,Orlando Wiet


In [6]:
df_stats.head()

,id,name,nickname,division,record,status,place_of_birth,trains_at,fighting_style,octagon_debut,age,height,weight,reach,leg_reach,wins,losses,draws,wins_by_knockout,first_round_finishes,win_by_dec,win_by_sub,sig_strikes_landed,sig_strikes_attempted,takedowns_landed,takedowns_attempted,sig_strikes_landed_per_minute,sig_strikes_absorbed_per_minute,takedowns_avg,submission_avg,sig_strikes_defense,takedown_defense,knockdown_avg,fight_time_avg,sig_strikes_standing,sig_strikes_clinch,sig_strikes_ground,head_target,body_target,leg_target
0,1,AJ Cunningham,"""The Savage""",Bantamweight Division,11-5-0 (W-L-D),Active,"Phoenix, United States",Factory X,None,"Mar. 2, 2024",30.0,70.0,136.0,71.0,41.0,11,5,0,4.0,5.0,3.0,4.0,121.0,412.0,NaN,3.0,4.0,6.0,0.00,0.52,48.0,46.0,0.00,09:34,118.0,3.0,0.0,65.0,35.0,21.0
1,2,AJ Dobson,None,Middleweight Division,7-3-0 (W-L-D),Active,"Brooklyn, United States",None,Freestyle,"Feb. 12, 2022",31.0,73.0,185.0,76.0,43.5,7,3,0,3.0,5.0,2.0,2.0,231.0,500.0,NaN,8.0,4.0,5.0,1.67,0.28,46.0,65.0,0.28,10:47,180.0,27.0,24.0,142.0,31.0,58.0
2,3,AJ Fletcher,"""The Ghost""",Welterweight Division,10-3-0 (W-L-D),Active,"Baton Rouge, United States",None,Striker,"Mar. 12, 2022",26.0,70.0,183.0,67.0,37.0,10,3,0,4.0,6.0,1.0,4.0,163.0,329.0,4.0,14.0,3.0,5.0,1.54,0.93,47.0,33.0,0.31,09:43,138.0,12.0,13.0,113.0,18.0,32.0
3,4,Aalon Cruz,None,Lightweight Division,8-4-0 (W-L-D),Not Fighting,"Parrish, United States",None,MMA,"Jul. 30, 2019",33.0,72.0,155.0,78.0,42.0,8,4,0,2.0,NaN,4.0,2.0,134.0,343.0,NaN,7.0,8.0,9.0,0.00,0.00,59.0,NaN,0.85,05:54,132.0,1.0,1.0,83.0,36.0,15.0
4,5,Aaron Brink,None,Heavyweight Division,8-3-0 (W-L-D),Not Fighting,United States,None,None,"Nov. 17, 2000",48.0,75.0,231.0,NaN,NaN,8,3,0,NaN,NaN,0.0,0.0,11.0,26.0,NaN,3.0,3.0,6.0,0.00,0.00,57.0,NaN,0.00,01:51,2.0,9.0,0.0,10.0,1.0,0.0


Review Stats

In [7]:
df_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985 entries, 0 to 2984
Data columns (total 40 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               2985 non-null   int64  
 1   name                             2985 non-null   object 
 2   nickname                         1780 non-null   object 
 3   division                         2869 non-null   object 
 4   record                           2943 non-null   object 
 5   status                           2951 non-null   object 
 6   place_of_birth                   2815 non-null   object 
 7   trains_at                        957 non-null    object 
 8   fighting_style                   989 non-null    object 
 9   octagon_debut                    2985 non-null   object 
 10  age                              2722 non-null   float64
 11  height                           2712 non-null   float64
 12  weight              

In [8]:
df_stats.describe()

,id,age,height,weight,reach,leg_reach,wins,losses,draws,wins_by_knockout,first_round_finishes,win_by_dec,win_by_sub,sig_strikes_landed,sig_strikes_attempted,takedowns_landed,takedowns_attempted,sig_strikes_landed_per_minute,sig_strikes_absorbed_per_minute,takedowns_avg,submission_avg,sig_strikes_defense,takedown_defense,knockdown_avg,sig_strikes_standing,sig_strikes_clinch,sig_strikes_ground,head_target,body_target,leg_target
count,2985.000000,2722.000000,2712.000000,2880.000000,1887.000000,1610.000000,2985.000000,2985.000000,2985.000000,1174.000000,838.000000,2932.000000,2932.000000,2690.000000,2690.000000,890.000000,2413.000000,2699.000000,2699.000000,2699.000000,2699.000000,2687.000000,2268.000000,2699.000000,2932.000000,2932.000000,2932.000000,2932.000000,2932.000000,2932.000000
mean,1493.000000,36.250551,65.470501,157.530000,71.614202,39.917081,11.495812,4.398995,0.167504,6.310051,6.309069,2.474420,2.380628,247.360967,553.575836,7.317978,21.335267,3.147092,3.896628,1.467910,0.614542,51.990696,61.227954,0.290774,164.155866,31.451910,31.336630,143.173943,47.005116,36.765348
std,861.839602,7.103092,17.843536,51.748562,4.296083,2.612560,8.712726,3.742633,0.587586,4.048632,3.792867,3.463268,3.554672,312.511618,693.455997,10.368086,28.225576,1.676662,2.494483,1.676565,1.209010,10.466248,21.180710,0.611585,237.626448,48.158197,52.331009,198.841810,68.271806,56.342565
min,1.000000,19.000000,0.000000,0.000000,58.500000,32.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,4.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,747.000000,31.000000,67.000000,136.000000,69.000000,38.000000,6.000000,2.000000,0.000000,3.000000,4.000000,0.000000,0.000000,45.000000,109.000000,2.000000,4.000000,2.000000,3.000000,0.210000,0.000000,46.000000,47.750000,0.000000,17.000000,3.000000,0.000000,16.000000,5.000000,3.000000
50%,1493.000000,35.000000,70.000000,156.000000,72.000000,40.000000,11.000000,4.000000,0.000000,6.000000,6.000000,1.000000,1.000000,130.000000,285.000000,4.000000,11.000000,3.000000,3.000000,1.020000,0.250000,53.000000,62.000000,0.000000,74.000000,13.000000,11.000000,67.000000,21.000000,16.000000
75%,2239.000000,40.000000,73.000000,185.000000,74.500000,41.500000,16.000000,6.000000,0.000000,8.000000,8.000000,4.000000,4.000000,327.750000,737.500000,8.000000,26.000000,4.000000,5.000000,2.130000,0.820000,59.000000,75.000000,0.410000,216.000000,39.000000,39.000000,189.000000,61.000000,46.000000
max,2985.000000,80.000000,84.000000,415.000000,84.500000,74.000000,88.000000,24.000000,10.000000,28.000000,26.000000,36.000000,47.000000,3655.000000,7602.000000,90.000000,281.000000,14.000000,53.000000,24.110000,21.950000,100.000000,100.000000,12.330000,3222.000000,491.000000,523.000000,2363.000000,893.000000,508.000000


In [9]:
df_stats.isnull().sum()

id                                    0
name                                  0
nickname                           1205
division                            116
record                               42
status                               34
place_of_birth                      170
trains_at                          2028
fighting_style                     1996
octagon_debut                         0
age                                 263
height                              273
weight                              105
reach                              1098
leg_reach                          1375
wins                                  0
losses                                0
draws                                 0
wins_by_knockout                   1811
first_round_finishes               2147
win_by_dec                           53
win_by_sub                           53
sig_strikes_landed                  295
sig_strikes_attempted               295
takedowns_landed                   2095


Find Duplicates Names and Drop

In [10]:
df_stats[df_stats.duplicated('name', keep=False) == True]

,id,name,nickname,division,record,status,place_of_birth,trains_at,fighting_style,octagon_debut,age,height,weight,reach,leg_reach,wins,losses,draws,wins_by_knockout,first_round_finishes,win_by_dec,win_by_sub,sig_strikes_landed,sig_strikes_attempted,takedowns_landed,takedowns_attempted,sig_strikes_landed_per_minute,sig_strikes_absorbed_per_minute,takedowns_avg,submission_avg,sig_strikes_defense,takedown_defense,knockdown_avg,fight_time_avg,sig_strikes_standing,sig_strikes_clinch,sig_strikes_ground,head_target,body_target,leg_target
387,388,Bruno Silva,"""Bulldog""",Flyweight Division,14-7-2 (W-L-D),Active,"Piracicaba, Brazil",American Top Team,Grappler,"Sep. 7, 2019",35.0,64.0,125.0,65.0,35.0,14,7,2,6.0,4.0,4.0,4.0,353.0,713.0,2.0,46.0,4.0,5.0,1.79,0.16,50.0,60.0,0.98,10:15,287.0,24.0,42.0,196.0,80.0,77.0
388,389,Bruno Silva,"""Bulldog""",Flyweight Division,14-7-2 (W-L-D),Active,"Piracicaba, Brazil",American Top Team,Grappler,"Sep. 7, 2019",35.0,64.0,125.0,65.0,35.0,14,7,2,6.0,4.0,4.0,4.0,353.0,713.0,2.0,46.0,4.0,5.0,1.79,0.16,50.0,60.0,0.98,10:15,287.0,24.0,42.0,196.0,80.0,77.0
449,450,Casey Kenney,None,Bantamweight Division,16-4-1 (W-L-D),Active,"Portland, United States",None,MMA,"Jul. 18, 2017",32.0,67.0,136.0,70.0,36.0,16,4,1,2.0,NaN,9.0,5.0,670.0,1570.0,2.0,28.0,5.0,5.0,1.09,0.43,58.0,61.0,0.00,13:48,598.0,54.0,18.0,361.0,168.0,141.0
450,451,Casey Kenney,None,Bantamweight Division,16-4-1 (W-L-D),Active,"Portland, United States",None,MMA,"Jul. 18, 2017",32.0,67.0,136.0,70.0,36.0,16,4,1,2.0,NaN,9.0,5.0,670.0,1570.0,2.0,28.0,5.0,5.0,1.09,0.43,58.0,61.0,0.00,13:48,598.0,54.0,18.0,361.0,168.0,141.0
1372,1373,Joey Gomez,"""KO King""",Bantamweight Division,6-2-0 (W-L-D),Not Fighting,"Fairfax, United States",Team Link Hooksett,None,"Jan. 18, 2016",36.0,68.0,135.0,73.0,40.0,6,2,0,NaN,NaN,0.0,0.0,59.0,207.0,NaN,1.0,2.0,4.0,0.62,0.00,55.0,50.0,0.62,12:07,47.0,12.0,0.0,36.0,9.0,14.0
1373,1374,Joey Gomez,"""KO King""",Bantamweight Division,6-2-0 (W-L-D),Not Fighting,"Fairfax, United States",Team Link Hooksett,None,"Jan. 18, 2016",36.0,68.0,135.0,73.0,40.0,6,2,0,NaN,NaN,0.0,0.0,59.0,207.0,NaN,1.0,2.0,4.0,0.62,0.00,55.0,50.0,0.62,12:07,47.0,12.0,0.0,36.0,9.0,14.0
2367,2368,Richie Vaculik,"""Vas""",Flyweight Division,10-5-0 (W-L-D),Not Fighting,"Sydney, Australia",None,None,"Dec. 7, 2013",40.0,66.0,125.0,69.0,41.0,10,5,0,NaN,NaN,1.0,6.0,166.0,355.0,NaN,20.0,4.0,4.0,3.00,1.00,57.0,32.0,0.00,11:15,132.0,23.0,11.0,107.0,31.0,28.0
2368,2369,Richie Vaculik,"""Vas""",Flyweight Division,10-5-0 (W-L-D),Not Fighting,"Sydney, Australia",None,None,"Dec. 7, 2013",40.0,66.0,125.0,69.0,41.0,10,5,0,NaN,NaN,1.0,6.0,166.0,355.0,NaN,20.0,4.0,4.0,3.00,1.00,57.0,32.0,0.00,11:15,132.0,23.0,11.0,107.0,31.0,28.0


In [11]:
df_stats.drop_duplicates(subset=['name'], inplace=True)

Find missing values

In [12]:
missing_df = pd.DataFrame({
    'missing_count': df_stats.isnull().sum(),
    'missing_precent': round((df_stats.isnull().sum() / len(df_stats)) * 100, 2)
}).sort_values(by='missing_precent', ascending=False)

missing_df

,missing_count,missing_precent
first_round_finishes,2144,71.92
takedowns_landed,2093,70.21
trains_at,2026,67.96
fighting_style,1994,66.89
wins_by_knockout,1809,60.68
leg_reach,1375,46.13
nickname,1204,40.39
reach,1098,36.83
takedown_defense,717,24.05
takedowns_attempted,572,19.19


Remove null values with these columns. They are the new fighters that still do not have stats and a few very old fighters with also no stats

In [13]:
df_stats[df_stats[['sig_strikes_standing', 'sig_strikes_clinch', 'sig_strikes_ground', 'head_target', 'body_target', 'leg_target']].isnull().all(axis=1)]

,id,name,nickname,division,record,status,place_of_birth,trains_at,fighting_style,octagon_debut,age,height,weight,reach,leg_reach,wins,losses,draws,wins_by_knockout,first_round_finishes,win_by_dec,win_by_sub,sig_strikes_landed,sig_strikes_attempted,takedowns_landed,takedowns_attempted,sig_strikes_landed_per_minute,sig_strikes_absorbed_per_minute,takedowns_avg,submission_avg,sig_strikes_defense,takedown_defense,knockdown_avg,fight_time_avg,sig_strikes_standing,sig_strikes_clinch,sig_strikes_ground,head_target,body_target,leg_target
58,59,Alberta Cerra,None,None,None,Not Fighting,United States,None,None,"Jul. 30, 2025",NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
138,139,Alvaro Ivan Lopez Rodrigues,None,None,None,Not Fighting,Mexico,None,None,"Jul. 30, 2025",NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
149,150,Amir Aliakbari,None,Heavyweight Division,0-0-0 (W-L-D),Not Fighting,None,None,None,"Jul. 30, 2025",39.0,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
244,245,Artem Frolov,None,Middleweight Division,0-0-0 (W-L-D),Not Fighting,None,None,None,"Jul. 30, 2025",31.0,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
251,252,Asikeerbai Jinensibieke,None,None,None,Active,"Xinjiang, China",None,None,"Jun. 10, 2022",34.0,NaN,155.0,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
312,313,Bilyal Makhov,None,None,None,Retired,"USSR, Russia",None,None,"Jul. 30, 2025",35.0,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
448,449,Carrese Archer,"""One Punch""",None,None,Not Fighting,None,None,None,"Jul. 30, 2025",33.0,72.0,184.0,73.0,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
495,496,Chi Lewis-Parry,None,Heavyweight Division,0-0-0 (W-L-D),Not Fighting,"Hitchin, United Kingdom",None,None,"Jul. 30, 2025",39.0,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
562,563,Cody Belisle,"""Bodacious""",None,None,Active,None,None,None,"Jul. 30, 2025",30.0,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
593,594,Cory Corbin,"""Hamma""",None,None,Active,None,None,None,"Jul. 30, 2025",36.0,NaN,254.0,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_stats = df_stats[~df_stats[['sig_strikes_standing', 'sig_strikes_clinch', 'sig_strikes_ground', 'head_target', 'body_target', 'leg_target']].isnull().all(axis=1)]

Drop irrelevant columns that also have a high null percentage

In [15]:
df_stats.drop(columns=['id', 'nickname', 'division', 'record', 'status', 'place_of_birth', 'trains_at', 'fighting_style'], inplace=True)

Convert octagon_debut to datetime

In [16]:
df_stats['octagon_debut'] = pd.to_datetime(df_stats['octagon_debut'], format='%b. %d, %Y')
df_stats

,name,octagon_debut,age,height,weight,reach,leg_reach,wins,losses,draws,wins_by_knockout,first_round_finishes,win_by_dec,win_by_sub,sig_strikes_landed,sig_strikes_attempted,takedowns_landed,takedowns_attempted,sig_strikes_landed_per_minute,sig_strikes_absorbed_per_minute,takedowns_avg,submission_avg,sig_strikes_defense,takedown_defense,knockdown_avg,fight_time_avg,sig_strikes_standing,sig_strikes_clinch,sig_strikes_ground,head_target,body_target,leg_target
0,AJ Cunningham,2024-03-02,30.0,70.0,136.0,71.0,41.0,11,5,0,4.0,5.0,3.0,4.0,121.0,412.0,NaN,3.0,4.0,6.0,0.00,0.52,48.0,46.0,0.00,09:34,118.0,3.0,0.0,65.0,35.0,21.0
1,AJ Dobson,2022-02-12,31.0,73.0,185.0,76.0,43.5,7,3,0,3.0,5.0,2.0,2.0,231.0,500.0,NaN,8.0,4.0,5.0,1.67,0.28,46.0,65.0,0.28,10:47,180.0,27.0,24.0,142.0,31.0,58.0
2,AJ Fletcher,2022-03-12,26.0,70.0,183.0,67.0,37.0,10,3,0,4.0,6.0,1.0,4.0,163.0,329.0,4.0,14.0,3.0,5.0,1.54,0.93,47.0,33.0,0.31,09:43,138.0,12.0,13.0,113.0,18.0,32.0
3,Aalon Cruz,2019-07-30,33.0,72.0,155.0,78.0,42.0,8,4,0,2.0,NaN,4.0,2.0,134.0,343.0,NaN,7.0,8.0,9.0,0.00,0.00,59.0,NaN,0.85,05:54,132.0,1.0,1.0,83.0,36.0,15.0
4,Aaron Brink,2000-11-17,48.0,75.0,231.0,NaN,NaN,8,3,0,NaN,NaN,0.0,0.0,11.0,26.0,NaN,3.0,3.0,6.0,0.00,0.00,57.0,NaN,0.00,01:51,2.0,9.0,0.0,10.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2980,Zubaira Tukhugov,2014-02-15,32.0,68.0,157.5,68.0,38.5,20,6,1,7.0,7.0,12.0,1.0,357.0,894.0,7.0,41.0,3.0,3.0,2.13,0.00,64.0,100.0,0.40,12:31,302.0,24.0,31.0,312.0,36.0,9.0
2981,Zviad Lazishvili,2021-10-23,31.0,66.0,135.0,69.0,37.0,13,1,0,NaN,NaN,4.0,9.0,63.0,160.0,NaN,6.0,4.0,6.0,0.00,0.00,51.0,NaN,0.00,15:00,60.0,3.0,0.0,25.0,17.0,21.0
2982,Zygimantas Ramaska,2024-08-24,28.0,71.0,145.5,73.0,NaN,9,3,0,5.0,5.0,0.0,4.0,6.0,10.0,NaN,NaN,1.0,2.0,0.00,2.41,30.0,33.0,0.00,06:14,2.0,4.0,0.0,5.0,1.0,0.0
2983,Zak Cummings,2013-08-29,38.0,72.0,206.0,75.0,40.0,25,7,0,5.0,NaN,6.0,12.0,500.0,1394.0,7.0,25.0,3.0,3.0,0.62,0.62,55.0,67.0,0.71,11:17,403.0,59.0,38.0,302.0,93.0,105.0


In [17]:
missing_df = pd.DataFrame({
    'missing_count': df_stats.isnull().sum(),
    'missing_precent': round((df_stats.isnull().sum() / len(df_stats)) * 100, 2)
}).sort_values(by='missing_precent', ascending=False)

missing_df


,missing_count,missing_precent
first_round_finishes,2091,71.41
takedowns_landed,2040,69.67
wins_by_knockout,1756,59.97
leg_reach,1324,45.22
reach,1049,35.83
takedown_defense,664,22.68
takedowns_attempted,519,17.73
sig_strikes_defense,245,8.37
sig_strikes_attempted,242,8.27
sig_strikes_landed,242,8.27


Fill null values with 0 where it makes sense

In [18]:
# For example, it is not unusual for a fighter to have 0 first_round_finishes
columns = ['first_round_finishes', 'takedowns_landed', 'wins_by_knockout', 'takedowns_attempted', 'takedown_defense']

for col in columns:
    df_stats[col] = df_stats[col].fillna(0)

In [19]:
df_stats.isnull().sum()

name                                  0
octagon_debut                         0
age                                 240
height                              232
weight                               70
reach                              1049
leg_reach                          1324
wins                                  0
losses                                0
draws                                 0
wins_by_knockout                      0
first_round_finishes                  0
win_by_dec                            0
win_by_sub                            0
sig_strikes_landed                  242
sig_strikes_attempted               242
takedowns_landed                      0
takedowns_attempted                   0
sig_strikes_landed_per_minute       233
sig_strikes_absorbed_per_minute     233
takedowns_avg                       233
submission_avg                      233
sig_strikes_defense                 245
takedown_defense                      0
knockdown_avg                       233


Impute Missing columns

Weight and Height have some 0 values, this does not make sense as someone can not have a value of height be equal to 0, which will impact the imputation.
I will convert to null and after impute.

In [20]:
df_stats[df_stats['height'] == 0]
df_stats[df_stats['weight'] == 0]

,name,octagon_debut,age,height,weight,reach,leg_reach,wins,losses,draws,wins_by_knockout,first_round_finishes,win_by_dec,win_by_sub,sig_strikes_landed,sig_strikes_attempted,takedowns_landed,takedowns_attempted,sig_strikes_landed_per_minute,sig_strikes_absorbed_per_minute,takedowns_avg,submission_avg,sig_strikes_defense,takedown_defense,knockdown_avg,fight_time_avg,sig_strikes_standing,sig_strikes_clinch,sig_strikes_ground,head_target,body_target,leg_target
33,Adrian Serrano,2000-06-09,59.0,0.0,0.0,NaN,NaN,0,0,0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,00:00,0.0,0.0,0.0,0.0,0.0,0.0
51,Adriano Santos,1998-10-16,NaN,0.0,0.0,NaN,NaN,0,0,0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,00:00,0.0,0.0,0.0,0.0,0.0,0.0
59,Alberto Cerro Leon,1994-03-11,NaN,75.0,0.0,NaN,NaN,0,0,0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,00:00,0.0,0.0,0.0,0.0,0.0,0.0
75,Alex Andrade,2000-06-09,49.0,0.0,0.0,NaN,NaN,0,0,0,0.0,0.0,0.0,0.0,4.0,11.0,0.0,0.0,0.0,3.0,0.00,0.75,53.0,25.0,0.0,20:00,3.0,1.0,0.0,3.0,1.0,0.0
84,Alex Hunter,1997-07-27,NaN,0.0,0.0,NaN,NaN,0,0,0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,00:00,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2848,Valeri Ignatov,1999-03-05,NaN,70.0,0.0,NaN,NaN,0,0,0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,00:00,0.0,0.0,0.0,0.0,0.0,0.0
2886,Wallid Ismail,1997-02-07,55.0,70.0,0.0,NaN,NaN,0,0,0,0.0,0.0,0.0,0.0,30.0,96.0,0.0,14.0,1.0,1.0,1.85,0.46,45.0,0.0,0.0,16:15,1.0,0.0,29.0,27.0,3.0,0.0
2901,Wes Albritton,1997-05-30,NaN,0.0,0.0,NaN,NaN,0,0,0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,00:00,0.0,0.0,0.0,0.0,0.0,0.0
2947,Yoshiki Takahashi,1997-02-07,54.0,0.0,0.0,NaN,NaN,0,0,0,0.0,0.0,0.0,0.0,15.0,29.0,0.0,1.0,2.0,6.0,0.00,2.26,45.0,67.0,0.0,02:13,6.0,0.0,9.0,11.0,1.0,3.0


In [21]:
df_stats[['height', 'weight']] = df_stats[['height', 'weight']].replace(0, np.nan)

In [22]:
print(df_stats.columns[df_stats.isnull().any()])
print("\n", len(df_stats.columns[df_stats.isnull().any()]))

Index(['age', 'height', 'weight', 'reach', 'leg_reach', 'sig_strikes_landed',
       'sig_strikes_attempted', 'sig_strikes_landed_per_minute',
       'sig_strikes_absorbed_per_minute', 'takedowns_avg', 'submission_avg',
       'sig_strikes_defense', 'knockdown_avg'],
      dtype='object')

 13


Impute all null values

In [23]:
missing_cols = df_stats.columns[df_stats.isnull().any()]

imputer = KNNImputer(n_neighbors=5)
df_stats[missing_cols] = imputer.fit_transform(df_stats[missing_cols])

In [24]:
df_stats.isnull().sum()

name                               0
octagon_debut                      0
age                                0
height                             0
weight                             0
reach                              0
leg_reach                          0
wins                               0
losses                             0
draws                              0
wins_by_knockout                   0
first_round_finishes               0
win_by_dec                         0
win_by_sub                         0
sig_strikes_landed                 0
sig_strikes_attempted              0
takedowns_landed                   0
takedowns_attempted                0
sig_strikes_landed_per_minute      0
sig_strikes_absorbed_per_minute    0
takedowns_avg                      0
submission_avg                     0
sig_strikes_defense                0
takedown_defense                   0
knockdown_avg                      0
fight_time_avg                     0
sig_strikes_standing               0
s

In [25]:
df_stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2928 entries, 0 to 2984
Data columns (total 32 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   name                             2928 non-null   object        
 1   octagon_debut                    2928 non-null   datetime64[ns]
 2   age                              2928 non-null   float64       
 3   height                           2928 non-null   float64       
 4   weight                           2928 non-null   float64       
 5   reach                            2928 non-null   float64       
 6   leg_reach                        2928 non-null   float64       
 7   wins                             2928 non-null   int64         
 8   losses                           2928 non-null   int64         
 9   draws                            2928 non-null   int64         
 10  wins_by_knockout                 2928 non-null   float64       
 

In [26]:
df_stats.columns

Index(['name', 'octagon_debut', 'age', 'height', 'weight', 'reach',
       'leg_reach', 'wins', 'losses', 'draws', 'wins_by_knockout',
       'first_round_finishes', 'win_by_dec', 'win_by_sub',
       'sig_strikes_landed', 'sig_strikes_attempted', 'takedowns_landed',
       'takedowns_attempted', 'sig_strikes_landed_per_minute',
       'sig_strikes_absorbed_per_minute', 'takedowns_avg', 'submission_avg',
       'sig_strikes_defense', 'takedown_defense', 'knockdown_avg',
       'fight_time_avg', 'sig_strikes_standing', 'sig_strikes_clinch',
       'sig_strikes_ground', 'head_target', 'body_target', 'leg_target'],
      dtype='object')

Change floats back to int at appropriate columns

In [27]:
cols_to_fix = ['age', 'wins', 'losses', 'draws', 'wins_by_knockout', 'first_round_finishes', 'win_by_dec',
            'win_by_sub', 'sig_strikes_landed', 'sig_strikes_attempted', 'takedowns_landed', 'takedowns_attempted',
            'sig_strikes_landed_per_minute', 'sig_strikes_absorbed_per_minute', 'sig_strikes_standing', 'sig_strikes_clinch', 'sig_strikes_ground',
            'head_target', 'body_target', 'leg_target']

for col in cols_to_fix:
    df_stats[col] = df_stats[col].round().astype('Int64')

Remove Fighters with octagon_debut before 2007

The reason for this is because the sport has changed significantly since then and does not represent the same type of stats we expect from today's fihgters.  <br>
Rules, fighters, fighting styles, techniques have all evolved.  <br>
Why 2007?  <br>
The simple answer is there are a handful of fighters that are still active and have been fighting since this year onwards.

In [28]:
df_stats = df_stats[df_stats['octagon_debut'] >= '2007-01-01']

In [ ]:
df_stats = df_stats.reset_index(drop=True)

df_stats

,name,octagon_debut,age,height,weight,reach,leg_reach,wins,losses,draws,wins_by_knockout,first_round_finishes,win_by_dec,win_by_sub,sig_strikes_landed,sig_strikes_attempted,takedowns_landed,takedowns_attempted,sig_strikes_landed_per_minute,sig_strikes_absorbed_per_minute,takedowns_avg,submission_avg,sig_strikes_defense,takedown_defense,knockdown_avg,fight_time_avg,sig_strikes_standing,sig_strikes_clinch,sig_strikes_ground,head_target,body_target,leg_target
0,AJ Cunningham,2024-03-02,30,70.0,136.0,71.0,41.0,11,5,0,4,5,3,4,121,412,0,3,4,6,0.00,0.52,48.0,46.0,0.00,09:34,118,3,0,65,35,21
1,AJ Dobson,2022-02-12,31,73.0,185.0,76.0,43.5,7,3,0,3,5,2,2,231,500,0,8,4,5,1.67,0.28,46.0,65.0,0.28,10:47,180,27,24,142,31,58
2,AJ Fletcher,2022-03-12,26,70.0,183.0,67.0,37.0,10,3,0,4,6,1,4,163,329,4,14,3,5,1.54,0.93,47.0,33.0,0.31,09:43,138,12,13,113,18,32
3,Aalon Cruz,2019-07-30,33,72.0,155.0,78.0,42.0,8,4,0,2,0,4,2,134,343,0,7,8,9,0.00,0.00,59.0,0.0,0.85,05:54,132,1,1,83,36,15
4,Aaron Jeffery,2025-07-30,30,74.0,185.5,73.5,41.0,6,2,0,0,0,0,0,57,125,0,4,3,4,0.00,0.00,34.0,50.0,0.00,09:12,23,34,0,30,18,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2465,Zubaira Tukhugov,2014-02-15,32,68.0,157.5,68.0,38.5,20,6,1,7,7,12,1,357,894,7,41,3,3,2.13,0.00,64.0,100.0,0.40,12:31,302,24,31,312,36,9
2466,Zviad Lazishvili,2021-10-23,31,66.0,135.0,69.0,37.0,13,1,0,0,0,4,9,63,160,0,6,4,6,0.00,0.00,51.0,0.0,0.00,15:00,60,3,0,25,17,21
2467,Zygimantas Ramaska,2024-08-24,28,71.0,145.5,73.0,38.8,9,3,0,5,5,0,4,6,10,0,0,1,2,0.00,2.41,30.0,33.0,0.00,06:14,2,4,0,5,1,0
2468,Zak Cummings,2013-08-29,38,72.0,206.0,75.0,40.0,25,7,0,5,0,6,12,500,1394,7,25,3,3,0.62,0.62,55.0,67.0,0.71,11:17,403,59,38,302,93,105


Round float columns

In [30]:
cols_to_fix = ['height', 'weight', 'reach', 'leg_reach', 'knockdown_avg']

for col in cols_to_fix:
    df_stats[col] = df_stats[col].round(2)

Concat df_stats & df_events

In [31]:
df_events.head()

,id,event_date,event_name,fighter_blue,fighter_red,knockdowns_blue,knockdowns_red,sig_attempts_blue,sig_attempts_red,sig_strikes_blue,sig_strikes_red,total_strikes_attempts_blue,total_strikes_attempts_red,total_strikes_blue,total_strikes_red,sub_attempts_blue,sub_attempts_red,takedowns_blue,takedowns_red,takedown_attempts_blue,takedown_attempts_red,control_time_blue,control_time_red,round,time,weight_class,win_method,winner
0,1,1994-03-11,UFC 2: No Way Out,Remco Pardoel,Royce Gracie,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,2,0,0,1,91,Open Weight,Submission,Royce Gracie
1,2,1994-03-11,UFC 2: No Way Out,Ray Wizard,Patrick Smith,0,0,1,1,1,1,2,1,2,1,0,1,0,0,0,1,0,0,1,58,Open Weight,Submission,Patrick Smith
2,3,1994-03-11,UFC 2: No Way Out,David Levicki,Johnny Rhodes,0,0,5,17,4,11,102,86,95,74,0,0,0,1,0,1,0,0,1,733,Open Weight,KO/TKO,Johnny Rhodes
3,4,1994-03-11,UFC 2: No Way Out,Sean Daugherty,Scott Morris,0,0,4,1,0,1,5,2,1,2,0,1,0,1,0,1,0,0,1,20,Open Weight,Submission,Scott Morris
4,5,1994-03-11,UFC 2: No Way Out,Robert Lucarelli,Orlando Wiet,0,0,6,12,2,8,6,15,2,11,1,0,1,0,1,0,0,0,1,170,Open Weight,KO/TKO,Orlando Wiet


Drop stat columns from each event

In [32]:
df_events.drop(columns=['id', 'knockdowns_blue', 'knockdowns_red', 'sig_attempts_blue', 'sig_attempts_red', 'sig_strikes_blue', 'sig_strikes_red', 'total_strikes_attempts_blue', 'total_strikes_attempts_red', 
'total_strikes_blue', 'total_strikes_red', 'sub_attempts_blue', 'sub_attempts_red', 'takedowns_blue', 'takedowns_red', 'takedown_attempts_blue', 'takedown_attempts_red', 'control_time_blue', 'control_time_red',
'round', 'time'], inplace=True)
df_events

,event_date,event_name,fighter_blue,fighter_red,weight_class,win_method,winner
0,1994-03-11,UFC 2: No Way Out,Remco Pardoel,Royce Gracie,Open Weight,Submission,Royce Gracie
1,1994-03-11,UFC 2: No Way Out,Ray Wizard,Patrick Smith,Open Weight,Submission,Patrick Smith
2,1994-03-11,UFC 2: No Way Out,David Levicki,Johnny Rhodes,Open Weight,KO/TKO,Johnny Rhodes
3,1994-03-11,UFC 2: No Way Out,Sean Daugherty,Scott Morris,Open Weight,Submission,Scott Morris
4,1994-03-11,UFC 2: No Way Out,Robert Lucarelli,Orlando Wiet,Open Weight,KO/TKO,Orlando Wiet
...,...,...,...,...,...,...,...
8250,2025-07-26,UFC Fight Night: Whittaker vs. De Ridder,Bogdan Guskov,Nikita Krylov,Light Heavyweight,KO/TKO,Bogdan Guskov
8251,2025-07-26,UFC Fight Night: Whittaker vs. De Ridder,Marc-Andre Barriault,Shara Magomedov,Middleweight,Decision - Unanimous,Shara Magomedov
8252,2025-07-26,UFC Fight Night: Whittaker vs. De Ridder,Jose Ochoa,Asu Almabayev,Flyweight,Decision - Unanimous,Asu Almabayev
8253,2025-07-26,UFC Fight Night: Whittaker vs. De Ridder,Carlos Leal,Muslim Salikhov,Welterweight,KO/TKO,Muslim Salikhov


In [33]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8255 entries, 0 to 8254
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   event_date    8255 non-null   object
 1   event_name    8255 non-null   object
 2   fighter_blue  8255 non-null   object
 3   fighter_red   8255 non-null   object
 4   weight_class  8255 non-null   object
 5   win_method    8255 non-null   object
 6   winner        8108 non-null   object
dtypes: object(7)
memory usage: 451.6+ KB


Keep dates equal and larger than the octagon_debut

In [34]:
df_events['event_date'] = pd.to_datetime(df_events['event_date'], errors='coerce')

min_date = df_stats['octagon_debut'].min()
df_events = df_events[df_events['event_date'] >= min_date]
df_events.reset_index(drop=True, inplace=True)

In [35]:
df_events.columns

Index(['event_date', 'event_name', 'fighter_blue', 'fighter_red',
       'weight_class', 'win_method', 'winner'],
      dtype='object')

In [36]:
df_stats.columns

Index(['name', 'octagon_debut', 'age', 'height', 'weight', 'reach',
       'leg_reach', 'wins', 'losses', 'draws', 'wins_by_knockout',
       'first_round_finishes', 'win_by_dec', 'win_by_sub',
       'sig_strikes_landed', 'sig_strikes_attempted', 'takedowns_landed',
       'takedowns_attempted', 'sig_strikes_landed_per_minute',
       'sig_strikes_absorbed_per_minute', 'takedowns_avg', 'submission_avg',
       'sig_strikes_defense', 'takedown_defense', 'knockdown_avg',
       'fight_time_avg', 'sig_strikes_standing', 'sig_strikes_clinch',
       'sig_strikes_ground', 'head_target', 'body_target', 'leg_target'],
      dtype='object')

Left Join events by both fighter stats of each event fight

In [37]:
# First, ensure the fighter names are clean for merging
df_events.loc[:, 'fighter_blue'] = df_events['fighter_blue'].str.strip()
df_events.loc[:, 'fighter_red'] = df_events['fighter_red'].str.strip()
df_stats.loc[:, 'name'] = df_stats['name'].str.strip()

# Merge stats for fighter_blue
df_merged = df_events.merge(
    df_stats,
    how='left',
    left_on='fighter_blue',
    right_on='name',
    suffixes=('', '_drop') 
)

# Rename fighter_blue stats columns with _blue suffix (excluding the original 'name')
cols_to_rename_blue = [col for col in df_stats.columns if col != 'name']
df_merged.rename(columns={col: f"{col}_blue" for col in cols_to_rename_blue}, inplace=True)
df_merged.drop(columns='name', inplace=True)

# Merge stats for fighter_red
df_merged = df_merged.merge(
    df_stats,
    how='left',
    left_on='fighter_red',
    right_on='name',
    suffixes=('', '_drop') 
)

# Rename fighter_red stats columns with _red suffix
cols_to_rename_red = [col for col in df_stats.columns if col != 'name']
df_merged.rename(columns={col: f"{col}_red" for col in cols_to_rename_red}, inplace=True)
df_merged.drop(columns='name', inplace=True)

In [38]:
df_merged

,event_date,event_name,fighter_blue,fighter_red,weight_class,win_method,winner,octagon_debut_blue,age_blue,height_blue,weight_blue,reach_blue,leg_reach_blue,wins_blue,losses_blue,draws_blue,wins_by_knockout_blue,first_round_finishes_blue,win_by_dec_blue,win_by_sub_blue,sig_strikes_landed_blue,sig_strikes_attempted_blue,takedowns_landed_blue,takedowns_attempted_blue,sig_strikes_landed_per_minute_blue,sig_strikes_absorbed_per_minute_blue,takedowns_avg_blue,submission_avg_blue,sig_strikes_defense_blue,takedown_defense_blue,knockdown_avg_blue,fight_time_avg_blue,sig_strikes_standing_blue,sig_strikes_clinch_blue,sig_strikes_ground_blue,head_target_blue,body_target_blue,leg_target_blue,octagon_debut_red,age_red,height_red,weight_red,reach_red,leg_reach_red,wins_red,losses_red,draws_red,wins_by_knockout_red,first_round_finishes_red,win_by_dec_red,win_by_sub_red,sig_strikes_landed_red,sig_strikes_attempted_red,takedowns_landed_red,takedowns_attempted_red,sig_strikes_landed_per_minute_red,sig_strikes_absorbed_per_minute_red,takedowns_avg_red,submission_avg_red,sig_strikes_defense_red,takedown_defense_red,knockdown_avg_red,fight_time_avg_red,sig_strikes_standing_red,sig_strikes_clinch_red,sig_strikes_ground_red,head_target_red,body_target_red,leg_target_red
0,2007-01-25,UFC Fight Night: Evans vs Salmon,Chad Reiner,Joshua Burkman,Welterweight,Decision - Unanimous,Joshua Burkman,2007-01-25,41,71.0,171.0,72.4,40.2,13,2,0,0,0,0,0,8,39,0,3,1,2,0.99,1.97,46.0,25.0,0.00,07:37,5,3,0,4,2,2,NaT,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2007-01-25,UFC Fight Night: Evans vs Salmon,Heath Herring,Jake O'Brien,Heavyweight,Decision - Unanimous,Jake O'Brien,2007-01-25,45,76.0,250.0,77.7,41.8,28,14,0,0,0,0,0,583,1157,0,40,2,2,1.22,0.73,55.0,33.0,0.37,12:19,220,66,297,302,158,123,NaT,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,2007-01-25,UFC Fight Night: Evans vs Salmon,Dean Lister,Nate Marquardt,Middleweight,Decision - Unanimous,Nate Marquardt,NaT,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,2007-01-25,UFC Fight Night: Evans vs Salmon,Sean Salmon,Rashad Evans,Light Heavyweight,KO/TKO,Rashad Evans,2007-01-25,45,70.0,205.0,76.0,43.0,9,3,0,0,0,0,0,11,33,0,4,1,2,4.06,0.00,65.0,0.0,0.00,02:28,2,2,7,10,1,0,NaT,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,2007-01-25,UFC Fight Night: Evans vs Salmon,Chris Price,Ed Herman,Middleweight,Submission,Ed Herman,NaT,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7593,2025-07-26,UFC Fight Night: Whittaker vs. De Ridder,Bogdan Guskov,Nikita Krylov,Light Heavyweight,KO/TKO,Bogdan Guskov,2023-09-02,32,75.0,205.5,76.0,41.0,18,3,0,15,13,0,3,58,113,0,0,4,3,0.00,0.00,50.0,50.0,0.00,15:00,49,9,0,25,25,8,2013-09-01,33,75.0,206.0,77.5,44.5,30,11,0,12,23,2,16,566,1035,1,50,4,3,2.17,1.37,45.0,53.0,0.57,06:34,299,145,122,360,132,74
7594,2025-07-26,UFC Fight Night: Whittaker vs. De Ridder,Marc-Andre Barriault,Shara Magomedov,Middleweight,Decision - Unanimous,Shara Magomedov,2025-07-30,35,73.0,185.0,74.

In [39]:
df_merged.isnull().sum()

event_date                   0
event_name                   0
fighter_blue                 0
fighter_red                  0
weight_class                 0
                          ... 
sig_strikes_clinch_red    1123
sig_strikes_ground_red    1123
head_target_red           1123
body_target_red           1123
leg_target_red            1123
Length: 69, dtype: int64

Drop rows where fighters have null stats, these rows are from fighters that have not been found during the join and have been assigned a null value

In [40]:
df_merged.columns

Index(['event_date', 'event_name', 'fighter_blue', 'fighter_red',
       'weight_class', 'win_method', 'winner', 'octagon_debut_blue',
       'age_blue', 'height_blue', 'weight_blue', 'reach_blue',
       'leg_reach_blue', 'wins_blue', 'losses_blue', 'draws_blue',
       'wins_by_knockout_blue', 'first_round_finishes_blue', 'win_by_dec_blue',
       'win_by_sub_blue', 'sig_strikes_landed_blue',
       'sig_strikes_attempted_blue', 'takedowns_landed_blue',
       'takedowns_attempted_blue', 'sig_strikes_landed_per_minute_blue',
       'sig_strikes_absorbed_per_minute_blue', 'takedowns_avg_blue',
       'submission_avg_blue', 'sig_strikes_defense_blue',
       'takedown_defense_blue', 'knockdown_avg_blue', 'fight_time_avg_blue',
       'sig_strikes_standing_blue', 'sig_strikes_clinch_blue',
       'sig_strikes_ground_blue', 'head_target_blue', 'body_target_blue',
       'leg_target_blue', 'octagon_debut_red', 'age_red', 'height_red',
       'weight_red', 'reach_red', 'leg_reach_red', 'wi

In [42]:
df_merged = df_merged.dropna(subset=['octagon_debut_blue',
    'age_blue', 'height_blue', 'weight_blue', 'reach_blue',
    'leg_reach_blue', 'wins_blue', 'losses_blue', 'draws_blue',
    'wins_by_knockout_blue', 'first_round_finishes_blue', 'win_by_dec_blue',
    'win_by_sub_blue', 'sig_strikes_landed_blue',
    'sig_strikes_attempted_blue', 'takedowns_landed_blue',
    'takedowns_attempted_blue', 'sig_strikes_landed_per_minute_blue',
    'sig_strikes_absorbed_per_minute_blue', 'takedowns_avg_blue',
    'submission_avg_blue', 'sig_strikes_defense_blue',
    'takedown_defense_blue', 'knockdown_avg_blue', 'fight_time_avg_blue',
    'sig_strikes_standing_blue', 'sig_strikes_clinch_blue',
    'sig_strikes_ground_blue', 'head_target_blue', 'body_target_blue',
    'leg_target_blue', 'octagon_debut_red',
    'age_red', 'height_red', 'weight_red', 'reach_red', 'leg_reach_red',
    'wins_red', 'losses_red', 'draws_red', 'wins_by_knockout_red',
    'first_round_finishes_red', 'win_by_dec_red', 'win_by_sub_red',
    'sig_strikes_landed_red', 'sig_strikes_attempted_red',
    'takedowns_landed_red', 'takedowns_attempted_red',
    'sig_strikes_landed_per_minute_red',
    'sig_strikes_absorbed_per_minute_red', 'takedowns_avg_red',
    'submission_avg_red', 'sig_strikes_defense_red', 'takedown_defense_red',
    'knockdown_avg_red', 'fight_time_avg_red', 'sig_strikes_standing_red',
    'sig_strikes_clinch_red', 'sig_strikes_ground_red', 'head_target_red',
    'body_target_red', 'leg_target_red'])


In [43]:
df_merged = df_merged.sort_values(by=['event_date']).reset_index(drop=True)

In [44]:
df_merged

,event_date,event_name,fighter_blue,fighter_red,weight_class,win_method,winner,octagon_debut_blue,age_blue,height_blue,weight_blue,reach_blue,leg_reach_blue,wins_blue,losses_blue,draws_blue,wins_by_knockout_blue,first_round_finishes_blue,win_by_dec_blue,win_by_sub_blue,sig_strikes_landed_blue,sig_strikes_attempted_blue,takedowns_landed_blue,takedowns_attempted_blue,sig_strikes_landed_per_minute_blue,sig_strikes_absorbed_per_minute_blue,takedowns_avg_blue,submission_avg_blue,sig_strikes_defense_blue,takedown_defense_blue,knockdown_avg_blue,fight_time_avg_blue,sig_strikes_standing_blue,sig_strikes_clinch_blue,sig_strikes_ground_blue,head_target_blue,body_target_blue,leg_target_blue,octagon_debut_red,age_red,height_red,weight_red,reach_red,leg_reach_red,wins_red,losses_red,draws_red,wins_by_knockout_red,first_round_finishes_red,win_by_dec_red,win_by_sub_red,sig_strikes_landed_red,sig_strikes_attempted_red,takedowns_landed_red,takedowns_attempted_red,sig_strikes_landed_per_minute_red,sig_strikes_absorbed_per_minute_red,takedowns_avg_red,submission_avg_red,sig_strikes_defense_red,takedown_defense_red,knockdown_avg_red,fight_time_avg_red,sig_strikes_standing_red,sig_strikes_clinch_red,sig_strikes_ground_red,head_target_red,body_target_red,leg_target_red
0,2007-04-05,UFC Fight Night: Stevenson vs Guillard,Naoyuki Kotani,Thiago Tavares,Lightweight,Decision - Unanimous,Thiago Tavares,2007-04-05,37,68.0,156.0,72.1,39.8,33,13,7,0,0,3,25,55,177,0,18,1,3,0.83,0.83,50.0,47.0,0.00,09:00,40,4,11,30,4,21,2007-04-05,38,67.0,146.0,68.0,39.0,24,7,1,0,0,0,0,329,820,0,98,2,2,3.83,1.46,63.0,63.0,0.00,09:08,209,30,90,229,76,24
1,2007-04-21,UFC 70: Nations Collide,Matt Grice,Terry Etim,Lightweight,Submission,Terry Etim,2007-04-21,41,68.0,145.0,69.4,38.1,15,5,0,0,0,0,0,252,472,0,24,4,3,2.19,0.49,56.0,82.0,0.73,08:48,68,59,125,196,33,23,2007-04-21,37,73.0,155.0,73.0,39.0,15,5,0,0,0,0,0,229,639,0,10,2,3,0.43,1.42,67.0,39.0,0.28,09:37,199,27,3,110,25,94
2,2007-04-21,UFC 70: Nations Collide,Dennis Siver,Jess Liaudin,Welterweight,Submission,Jess Liaudin,2007-04-21,44,65.0,146.0,70.0,38.0,23,11,0,0,0,0,0,869,2678,13,40,4,3,0.87,0.33,65.0,65.0,0.47,10:41,737,46,86,494,118,257,2007-04-21,32,69.0,155.0,71.3,39.2,12,11,0,0,0,0,0,90,175,0,15,2,3,3.28,1.64,42.0,31.0,0.82,07:19,40,35,15,70,9,11
3,2007-06-12,UFC Fight Night: Stout vs Fisher,Luke Caudillo,Nate Mohr,Lightweight,Decision - Unanimous,Nate Mohr,2007-06-12,42,67.0,156.0,72.0,40.1,15,8,0,0,0,0,0,68,235,0,5,2,5,0.44,0.00,55.0,86.0,0.87,11:30,60,8,0,46,15,7,2007-04-05,40,69.0,156.0,71.5,39.4,8,6,0,0,0,0,0,126,301,0,2,4,3,0.45,0.00,75.0,71.0,0.00,08:19,110,9,7,83,15,28
4,2007-06-12,UFC Fight Night: Stout vs Fisher,Jason Black,Thiago Tavares,Lightweight,Submission,Thiago Tavares,2007-06-12,50,68.0,156.0,72.9,40.8,22,4,1,0,0,0,0,33,69,0,3,1,3,0.48,6.25,45.0,29.0,0.00,09:22,9,7,17,27,6,0,2007-04-05,38,67.0,146.0,68.0,39.0,24,7,1,0,0,0,0,329,820,0,98,2,2,3.83,1.46,63.0,63.0,0.00,09:08,209,30,90,229,76,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5828,2025-07-26,UFC Fight Night: Whittaker vs. De Ridder,Said Nurmagomedov,Bryce Mitchell,Bantamweight,Decision - Unanimous,Bryce Mitchell,2018-07-14,33,68.0,135.5,70.0,38.5,18,5,0,4,9,8,6,321,661,1,39,3,2,0.86,1.15,56.0,63.0,0.29,09:28,244,46,31,173,79,69,2018-07-06,30,70.0,146.0,70.0,40.0,18,4,0,1,8,8,9,315,530,3,83,2,2,3.24,1.19,59.0,33.0,0.11,11:35,176,24,115,183,90,42
5829,2025-07-26,UFC Fight Night: Whittaker vs. De Ridder,Marcus McGhee,Petr Yan,Bantamweight,Decision - Unanimous,Petr Yan,2023-04-29,35,68.0,135.6,69.0,38.0,10,2,0,8,0,1,1,263,582,0,7,6,4,0.31,0.31,58.0,90.0,0.94,09:34,242,10,11,181,42,40,2018-06-23,32,67.5,135.5,67.0,38.0,19,5,0,7,3,11,1,1317,2422,0,56,5,4,1.58,0.12,59.0,85.0,0.58,17:08,967,158,192,899,259,159
5830,20

Save to csv for further use

In [57]:
df_merged.to_csv("../data/merged_df.csv", index=False)